In [1]:
from pyspark.sql import SparkSession
import pyspark
from pyspark import SparkContext

sc =SparkContext.getOrCreate()
from pyspark.sql import SQLContext
trurl='/home/aaftab/Dropbox/Techsoc/train_techsoc.csv'
tsturl='/home/aaftab/Dropbox/Techsoc/test_techsoc.csv'

sc.addFile(trurl)
sc.addFile(tsturl)

sqlcontext=SQLContext(sc)

from pyspark import SparkFiles
trdf=sqlcontext.read.csv(SparkFiles.get('train_techsoc.csv'), header=True, inferSchema=True)
tstdf=sqlcontext.read.csv(SparkFiles.get('test_techsoc.csv'), header=True, inferSchema=True)

from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['x_sim','y_sim','z_sim','Vx_sim','Vy_sim','Vz_sim','sat_id','x_sim_ff','y_sim_ff','z_sim_ff','Vx_sim_ff','Vy_sim_ff','Vz_sim_ff','x_sim_bf','y_sim_bf','z_sim_bf','Vx_sim_bf','Vy_sim_bf','Vz_sim_bf'], outputCol = 'features')
trd=vectorAssembler.transform(trdf)

trdx=trd.select(['features','x'])
trdy=trd.select(['features','y'])
trdz=trd.select(['features','z'])

trdVx=trd.select(['features','Vx'])
trdVy=trd.select(['features','Vy'])
trdVz=trd.select(['features','Vz'])

from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols =['x_sim','y_sim','z_sim','Vx_sim','Vy_sim','Vz_sim','sat_id','x_sim_ff','y_sim_ff','z_sim_ff','Vx_sim_ff','Vy_sim_ff','Vz_sim_ff','x_sim_bf','y_sim_bf','z_sim_bf','Vx_sim_bf','Vy_sim_bf','Vz_sim_bf'], outputCol = 'features')
tstd=vectorAssembler.transform(tstdf)
tstd=tstd.select(['features'])


In [2]:
from pyspark.ml.feature import MinMaxScaler
# Let us create an object of MinMaxScaler class
MinMaxScalerizer=MinMaxScaler().setMin(5).setMax(10).setInputCol("features").setOutputCol("minmax_features")

transx=MinMaxScalerizer.fit(trdx).transform(trdx)
transy=MinMaxScalerizer.fit(trdy).transform(trdy)
transz=MinMaxScalerizer.fit(trdy).transform(trdz)

transVx=MinMaxScalerizer.fit(trdVx).transform(trdVx)
transVy=MinMaxScalerizer.fit(trdVy).transform(trdVy)
transVz=MinMaxScalerizer.fit(trdVz).transform(trdVz)

In [3]:
from pyspark.ml.feature import MinMaxScaler
# Let us create an object of MinMaxScaler class
MinMaxScalerizer=MinMaxScaler().setMin(5).setMax(10).setInputCol("features").setOutputCol("minmax_features")
transt=MinMaxScalerizer.fit(trd).transform(tstd)
transt.show(5)

+--------------------+--------------------+
|            features|     minmax_features|
+--------------------+--------------------+
|[-67817.397487337...|[7.48023212002414...|
|[-69478.899261576...|[7.47236845339599...|
|[-69610.589087625...|[7.47174518298700...|
|[-68136.264984711...|[7.47872296262402...|
|[-64959.510617139...|[7.49375811798372...|
+--------------------+--------------------+
only showing top 5 rows



In [4]:
from pyspark.ml.feature import PCA
pca = PCA(k=8, inputCol="minmax_features", outputCol="pcaFeatures")

modelx = pca.fit(transx)
transformedx=modelx.transform(transx)
modely = pca.fit(transy)
transformedy=modely.transform(transy)
modelz = pca.fit(transz)
transformedz=modelz.transform(transz)

modelVx = pca.fit(transVx)
transformedVx=modelVx.transform(transVx)
modelVy = pca.fit(transVy)
transformedVy=modelVy.transform(transVy)
modelVz = pca.fit(transVz)
transformedVz=modelVz.transform(transVz)


In [5]:
from pyspark.ml.feature import PCA
pca = PCA(k=8, inputCol="minmax_features", outputCol="pcaFeatures")
modelt = pca.fit(transt)
print(sum(modelt.explainedVariance))
transformedt=modelt.transform(transt)
transformedt.show(5)

0.9815558570645541
+--------------------+--------------------+--------------------+
|            features|     minmax_features|         pcaFeatures|
+--------------------+--------------------+--------------------+
|[-67817.397487337...|[7.48023212002414...|[-5.5218024783223...|
|[-69478.899261576...|[7.47236845339599...|[-5.5225038026794...|
|[-69610.589087625...|[7.47174518298700...|[-5.5237439867107...|
|[-68136.264984711...|[7.47872296262402...|[-5.5252047030163...|
|[-64959.510617139...|[7.49375811798372...|[-5.5269114341109...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [6]:
from pyspark.ml.regression import LinearRegression

rfx=LinearRegression(featuresCol = 'minmax_features', labelCol = 'x')
rfy=LinearRegression(featuresCol = 'minmax_features', labelCol = 'y')
rfz=LinearRegression(featuresCol = 'minmax_features', labelCol = 'z')

rfVx=LinearRegression(featuresCol = 'minmax_features', labelCol = 'Vx')
rfVy=LinearRegression(featuresCol = 'minmax_features', labelCol = 'Vy')
rfVz=LinearRegression(featuresCol = 'minmax_features', labelCol = 'Vz')

rfmodelx = rfx.fit(transformedx)
rfmodely = rfy.fit(transformedy)
rfmodelz = rfz.fit(transformedz)

rfmodelVx = rfVx.fit(transformedVx)
rfmodelVy = rfVy.fit(transformedVy)
rfmodelVz = rfVz.fit(transformedVz)

In [7]:
import pandas as pd

rfpredx=rfmodelx.transform(transformedt)
rfpredx.select(['prediction']).toPandas().to_csv('x.csv')
rfpredy=rfmodely.transform(transformedt)
rfpredy.select(['prediction']).toPandas().to_csv('y.csv')
rfpredz=rfmodelz.transform(transformedt)
rfpredz.select(['prediction']).toPandas().to_csv('z.csv')

rfpredVx=rfmodelVx.transform(transformedt)
rfpredVx.select(['prediction']).toPandas().to_csv('Vx.csv')
rfpredVy=rfmodelVy.transform(transformedt)
rfpredVy.select(['prediction']).toPandas().to_csv('Vy.csv')
rfpredVz=rfmodelVz.transform(transformedt)
rfpredVz.select(['prediction']).toPandas().to_csv('Vz.csv')

In [8]:
import pandas as pd

x = pd.read_csv('/home/aaftab/spark/spark-3.0.0-preview2-bin-hadoop2.7/x.csv')
y = pd.read_csv('/home/aaftab/spark/spark-3.0.0-preview2-bin-hadoop2.7/y.csv')
z= pd.read_csv('/home/aaftab/spark/spark-3.0.0-preview2-bin-hadoop2.7/z.csv')

Vx=pd.read_csv('/home/aaftab/spark/spark-3.0.0-preview2-bin-hadoop2.7/Vx.csv')
Vy=pd.read_csv('/home/aaftab/spark/spark-3.0.0-preview2-bin-hadoop2.7/Vy.csv')
Vz=pd.read_csv('/home/aaftab/spark/spark-3.0.0-preview2-bin-hadoop2.7/Vz.csv')

df3 = pd.concat([x,y,z,Vx,Vy,Vz], axis=1)
df3.to_csv('file3.csv', index=False)